In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from catboost import CatBoostRegressor, CatBoostClassifier

data_DIR = "../data/rapm/"
misc_DIR = "../data/misc/"
model_path = "../data/models/"
pbp_DIR = "../data/pbpdata/"
fig_DIR = "../figs/analysis/"

team_dict, team_list  = get_teams()

In [ ]:
dfw = pd.read_parquet(data_DIR + "NBA_rapm_possessions_odds_2017_2024.parquet")
dfw["spreadm"] = dfw["spread"]*(dfw["secs"]/48/60)**4
features = ['margin','secs','spreadm','period']
len(dfw)

In [ ]:
dfw.columns

## Features

In [ ]:
features = ['margin','secs','spreadm','period']
rr = 22

In [ ]:
test_gid = dfw['gid'].sample(frac=0.8, random_state=rr).to_list()
dfw11 = dfw[dfw['gid'].isin(test_gid)]
dfw12 = dfw11.query("secs <=120")
for i in range(4):
    dfw11  = pd.concat([dfw11,dfw12])
dfw1 = dfw11
dfw2 = dfw[dfw['gid'].isin(test_gid)]

X_train =  dfw1[features].values
y_train = dfw1['win'].values
X_test =  dfw2[features].values
y_test = dfw2['win'].values
X = dfw[features].values
y = dfw['win'].values

In [ ]:
X_train

## Catboost

In [ ]:
# model = CatBoostRegressor(learning_rate=1, depth=6, loss_function='RMSE')
model = CatBoostClassifier(loss_function="Logloss",task_type="GPU",learning_rate=0.1,depth=12)
fit_model = model.fit(X_train, y_train, plot=True, eval_set=(X_test,y_test),verbose=50)
# print(fit_model.get_params())

In [ ]:
ypred = model.predict_proba(X_test)

In [ ]:
i=0

In [ ]:
dfw2["wp"] = ypred[:,1]
gameids = dfw2.gid.unique()
# gid = gameids[np.random.randint(0,len(gameids))]
gid = gameids[i]
i+=1

In [ ]:
data = dfw2.query(f"gid == '{gid}'")
p = (
    ggplot(data)
    + aes(x="secs",y="wp")
    + geom_point()
    + geom_line()
    + scale_y_continuous(labels=percent_format())
    + scale_x_reverse()
    + theme_idv
)
p

In [ ]:
p1 = (
    ggplot(data)
    + aes(x="secs",y="margin")
    + geom_point()
    + geom_line()
    + scale_x_reverse()
    + theme_idv
)
p1

In [ ]:
gfhfg

In [ ]:
line =  5
dfw3 = dfw2.query(f"spread == {line}")
p = (
    ggplot(dfw3)
    + aes(x="secs",y="wp",group="gid")
    + geom_smooth(se=False,size=0.3)
    + scale_x_reverse()
    + scale_y_continuous(labels=percent_format())
    + theme_idv
    + labs(
        # title = f"{team_dict[dfw1["tida"].iloc[0]]} vs {team_dict[dfw1["tidh"].iloc[0]]}"
    )
)
p